In [1]:
#import libraries
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
#initialization of known entities
url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'
db_name = 'Movies.db'
table_name = 'Top_50'
csv_path = 'top-50-films.csv'
df = pd.DataFrame(columns=["Average Rank","Film","Year"])
count = 0

In [3]:
#load the webpage for webscraping
html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')

In [4]:
#scraping of information
tables = data.find_all('tbody')
rows = tables[0].find_all('tr')

In [5]:
#iterate over the contents of the rows
for row in rows:    
    #check for the loop counter to restrict to 50 entries
    if count<50:    
        #Extract all the td data objects in the row and save them to col
        col = row.find_all('td')    
        #Check if the length of col is 0
        if len(col)!=0: 
            #Create a dictionary data_dict with the keys same as the columns of the dataframe created
            data_dict = {"Average Rank": col[0].contents[0],
                         "Film": col[1].contents[0],
                         "Year": col[2].contents[0]}
            #Convert the dictionary to a dataframe and concatenate it with the existing one
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)
            #Increment the loop counter
            count+=1
    else:
        break

In [6]:
print(df)

   Average Rank                                           Film  Year
0             1                                  The Godfather  1972
1             2                                   Citizen Kane  1941
2             3                                     Casablanca  1942
3             4                         The Godfather, Part II  1974
4             5                            Singin' in the Rain  1952
5             6                                         Psycho  1960
6             7                                    Rear Window  1954
7             8                                 Apocalypse Now  1979
8             9                          2001: A Space Odyssey  1968
9            10                                  Seven Samurai  1954
10           11                                        Vertigo  1958
11           12                                    Sunset Blvd  1950
12           13                                   Modern Times  1936
13           14                   

In [7]:
#save the dataframe to a csv file
df.to_csv(csv_path)

storing the data into a database:
    1. initialize a connection to the database
    2. save the dataframe as a table
    3. close the connection

In [8]:
conn = sqlite3.connect(db_name)
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()